In [6]:
from transformers import AutoModel, AutoTokenizer
import torch
import torch.nn.functional as F
# Load PhoBERT model and tokenizer
model_name = "vinai/phobert-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Load the educational data (Vietnamese FAQs)
import json

with open('educational_data_vn.json', 'r') as f:
    educational_data = json.load(f)

In [7]:
# Function to compute embeddings using PhoBERT
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        model_output = model(**inputs)
    return model_output.last_hidden_state.mean(dim=1)  # Get the mean embedding for the sentence

# Function to find the most similar predefined question
def get_most_similar_question(question, data):
    question_embedding = get_embeddings(question)
    max_similarity = -1
    best_answer = "Xin lỗi, tôi không hiểu câu hỏi của bạn. Vui lòng thử lại."

    for category in data:
        for item in data[category]["questions"]:
            predefined_question = item["question"]
            predefined_embedding = get_embeddings(predefined_question)

            # Compute cosine similarity between the input question and predefined question
            similarity = F.cosine_similarity(question_embedding, predefined_embedding).item()

            # Find the question with the highest similarity
            if similarity > max_similarity:
                max_similarity = similarity
                best_answer = item["answer"]

    return best_answer

# Function to get chatbot response
def chatbot_response(input_text):
    response = get_most_similar_question(input_text, educational_data)
    return response

# Example usage
student_question = "Thong tin ve truong?"
response = chatbot_response(student_question)
print(response)

Trường Đại học Quốc tế là một trong bảy trường đại học thành viên trực thuộc Đại học Quốc gia Thành phố Hồ Chí Minh, được thành lập vào tháng 12 năm 2003. Đây là trường đại học công lập đa ngành đầu tiên và duy nhất hiện nay tại Việt Nam sử dụng hoàn toàn tiếng Anh trong công tác đào tạo giảng dạy và nghiên cứu.
Hiện nay, Nhà trường đang đào tạo hệ chính quy bậc đại học và sau đại học. Nhà trường tập trung đào tạo các ngành học thuộc lĩnh vực mũi nhọn như như kinh tế, quản lý, kỹ thuật công nghệ. Mô hình hoạt động của trường được xây dựng hiện đại theo tiêu chuẩn quốc tế về đội ngũ giảng viên, giáo trình, chương trình học có định hướng và liên thông với các trường đại học có thứ hạng cao ở các nước trên thế giới như Hoa Kỳ, Châu u và khu vực Châu Á – Thái Bình Dương.
Mục tiêu của Trường Đại học Quốc tế là trở thành một trong những trường đại học nghiên cứu hàng đầu tại Việt Nam và trong khu vực, có khả năng đào tạo và cung cấp nguồn nhân lực chất lượng cao nhằm đáp ứng được nhu cầu phá